# 1. Bibliothek laden

In [296]:
import pandas as pd
import plotly.graph_objects as go
import os
import numpy as np
from dash import Dash, dcc, html, Input, Output, callback
from dash import Dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from matplotlib import pyplot as plt
import matplotlib as mpl
import locale
import matplotlib.ticker as mticker

#Cheatsheets & Hilfe
#https://hackerthemes.com/bootstrap-cheatsheet/#mt-1
#Bootstrap Themes: https://bootswatch.com/flatly/

# 2. Daten laden & Measures einfügen

## Aktienkurse

In [314]:
folder_path = 'D:\Python\Visual Analytics\Data_stocks'  # Ordnerpfad allenfalls anpassen
data_frames = []

for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        data = pd.read_csv(file_path)
        filename = os.path.splitext(file)[0]  # Dateiname
        data['Stock/Index'] = filename  # Hinzufügen der Dateinamens-Spalte
        data_frames.append(data)

In [315]:
data_frames

[            Date          Open          High           Low         Close  \
 0     2018-09-25   6603.640137   6603.640137   6381.859863   6446.470215   
 1     2018-09-26   6452.790039   6585.910156   6397.890137   6495.000000   
 2     2018-09-27   6495.290039   6712.100098   6464.950195   6676.750000   
 3     2018-09-28   6678.750000   6785.029785   6598.319824   6644.129883   
 4     2018-09-29   6643.100098   6643.100098   6511.649902   6601.959961   
 ...          ...           ...           ...           ...           ...   
 1822  2023-09-21  27129.839844  27152.939453  26389.300781  26567.632813   
 1823  2023-09-22  26564.056641  26726.078125  26495.533203  26579.568359   
 1824  2023-09-23  26578.556641  26634.185547  26520.519531  26579.390625   
 1825  2023-09-24  26579.373047  26716.058594  26221.050781  26256.826172   
 1826  2023-09-25  26221.669922  26382.966797  26027.330078  26342.904297   
 
          Adj Close       Volume Stock/Index  
 0      6446.470215   47261

In [316]:
df_stocks = pd.concat(data_frames, ignore_index=True) # enthält die aggregierten Daten mit der zusätzlichen Information "Titel" in der letzten Spalte
print(df_stocks)

             Date         Open         High          Low        Close  \
0      2018-09-25  6603.640137  6603.640137  6381.859863  6446.470215   
1      2018-09-26  6452.790039  6585.910156  6397.890137  6495.000000   
2      2018-09-27  6495.290039  6712.100098  6464.950195  6676.750000   
3      2018-09-28  6678.750000  6785.029785  6598.319824  6644.129883   
4      2018-09-29  6643.100098  6643.100098  6511.649902  6601.959961   
...           ...          ...          ...          ...          ...   
11858  2023-09-19   147.604996   147.889999   147.514999   147.604996   
11859  2023-09-20   147.742004   148.162994   147.623001   147.742004   
11860  2023-09-21   148.320999   148.447998   147.324997   148.320999   
11861  2023-09-22   147.643997   148.404007   147.529007   147.643997   
11862  2023-09-25   148.360001   148.964996   148.210007   148.835999   

         Adj Close        Volume     Stock/Index  
0      6446.470215  4.726180e+09         BTC-USD  
1      6495.000000  4

In [317]:
df_stocks.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


In [318]:
df_stocks.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11863 entries, 0 to 11862
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         11863 non-null  object 
 1   Open         11863 non-null  float64
 2   High         11863 non-null  float64
 3   Low          11863 non-null  float64
 4   Close        11863 non-null  float64
 5   Adj Close    11863 non-null  float64
 6   Volume       11863 non-null  float64
 7   Stock/Index  11863 non-null  object 
dtypes: float64(6), object(2)
memory usage: 741.6+ KB


In [319]:
# Datum formatieren und Infomationen hinzufügen
df_stocks['Date'] = pd.to_datetime(df_stocks['Date'], format='%Y-%m-%d')
df_stocks['Month'] = pd.to_datetime(df_stocks['Date']).dt.month
df_stocks['Year'] = pd.to_datetime(df_stocks['Date']).dt.year
df_stocks['MonthYear'] = df_stocks['Date'].dt.to_period('M')
df_stocks['Open'] = df_stocks['Open']
df_stocks['Close'] = df_stocks['Close']
df_stocks['High'] = df_stocks['High']
df_stocks['Low'] = df_stocks['Low']
df_stocks['Adj Close'] = df_stocks['Adj Close']
df_stocks['Volume'] = df_stocks['Volume'].round().astype(int)

df_stocks

,Date,Open,High,Low,Close,Adj Close,Volume,Stock/Index,Month,Year,MonthYear
0,2018-09-25,6603.640137,6603.640137,6381.859863,6446.470215,6446.470215,-2147483648,BTC-USD,9,2018,2018-09
1,2018-09-26,6452.790039,6585.910156,6397.890137,6495.000000,6495.000000,-2147483648,BTC-USD,9,2018,2018-09
2,2018-09-27,6495.290039,6712.100098,6464.950195,6676.750000,6676.750000,-2147483648,BTC-USD,9,2018,2018-09
3,2018-09-28,6678.750000,6785.029785,6598.319824,6644.129883,6644.129883,-2147483648,BTC-USD,9,2018,2018-09
4,2018-09-29,6643.100098,6643.100098,6511.649902,6601.959961,6601.959961,-2147483648,BTC-USD,9,2018,2018-09
...,...,...,...,...,...,...,...,...,...,...,...
11858,2023-09-19,147.604996,147.889999,147.514999,147.604996,147.604996,0,USDJPY (JPY=X),9,2023,2023-09
11859,2023-09-20,147.742004,148.162994,147.623001,147.742004,147.742004,0,USDJPY (JPY=X),9,2023,2023-09
11860,2023-09-21,148.320999,148.447998,147.324997,148.320999,148.320999,0,USDJPY (JPY=X),9,2023,2023-09
11861,2023-09-22,147.643997,148.404007,147.529007,147.643997,147.643997,0,USDJPY (JPY=X),9,2023,2023-09


In [320]:
# neue Measures hinzufügen
df_stocks['Return'] = (df_stocks['Close'] - df_stocks['Open'])
df_stocks['Return %'] = ((df_stocks['Return'] / df_stocks['Open']) *100).round(1)
# Monatszahlen
df_stocks['Monthly Return'] = df_stocks.groupby(['Stock/Index', 'MonthYear'])['Return'].transform('sum')
df_stocks['Open Value Month'] = df_stocks.groupby(['Stock/Index', 'MonthYear'])['Open'].transform('first') # wird nur für die Kalkulation benötigt
df_stocks['Monthly Return %'] = ((df_stocks['Monthly Return'] / df_stocks['Open Value Month']) *100).round(1)
# Jahreszahlen
df_stocks['Yearly Return'] = df_stocks.groupby(['Stock/Index', 'Year'])['Return'].transform('sum')
df_stocks['Open Value Year'] = df_stocks.groupby(['Stock/Index', 'Year'])['Open'].transform('first') # wird nur für die Kalkulation benötigt
df_stocks['Yearly Return %'] = ((df_stocks['Yearly Return'] / df_stocks['Open Value Year']) *100).round(1)

df_stocks

,Date,Open,High,Low,Close,Adj Close,Volume,Stock/Index,Month,Year,MonthYear,Return,Return %,Monthly Return,Open Value Month,Monthly Return %,Yearly Return,Open Value Year,Yearly Return %
0,2018-09-25,6603.640137,6603.640137,6381.859863,6446.470215,6446.470215,-2147483648,BTC-USD,9,2018,2018-09,-157.169922,-2.4,11.589844,6603.640137,0.2,-2863.778316,6603.640137,-43.4
1,2018-09-26,6452.790039,6585.910156,6397.890137,6495.000000,6495.000000,-2147483648,BTC-USD,9,2018,2018-09,42.209961,0.7,11.589844,6603.640137,0.2,-2863.778316,6603.640137,-43.4
2,2018-09-27,6495.290039,6712.100098,6464.950195,6676.750000,6676.750000,-2147483648,BTC-USD,9,2018,2018-09,181.459961,2.8,11.589844,6603.640137,0.2,-2863.778316,6603.640137,-43.4
3,2018-09-28,6678.750000,6785.029785,6598.319824,6644.129883,6644.129883,-2147483648,BTC-USD,9,2018,2018-09,-34.620117,-0.5,11.589844,6603.640137,0.2,-2863.778316,6603.640137,-43.4
4,2018-09-29,6643.100098,6643.100098,6511.649902,6601.959961,6601.959961,-2147483648,BTC-USD,9,2018,2018-09,-41.140137,-0.6,11.589844,6603.640137,0.2,-2863.778316,6603.640137,-43.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11858,2023-09-19,147.604996,147.889999,147.514999,147.604996,147.604996,0,USDJPY (JPY=X),9,2023,2023-09,0.000000,0.0,0.475998,145.473007,0.3,0.445008,130.845001,0.3
11859,2023-09-20,147.742004,148.162994,147.623001,147.742004,147.742004,0,USDJPY (JPY=X),9,2023,2023-09,0.000000,0.0,0.475998,145.473007,0.3,0.445008,130.845001,0.3
11860,2023-09-21,148.320999,148.447998,147.324997,148.320999,148.320999,0,USDJPY (JPY=X),9,2023,2023-09,0.000000,0.0,0.475998,145.473007,0.3,0.445008,130.845001,0.3
11861,2023-09-22,147.643997,148.404007,147.529007,147.643997,147.643997,0,USDJPY (JPY=X),9,2023,2023-09,0.000000,0.0,0.475998,145.473007,0.3,0.445008,130.845001,0.3


In [322]:
df_stocks.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11863 entries, 0 to 11862
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              11863 non-null  datetime64[ns]
 1   Open              11863 non-null  float64       
 2   High              11863 non-null  float64       
 3   Low               11863 non-null  float64       
 4   Close             11863 non-null  float64       
 5   Adj Close         11863 non-null  float64       
 6   Volume            11863 non-null  int32         
 7   Stock/Index       11863 non-null  object        
 8   Month             11863 non-null  int32         
 9   Year              11863 non-null  int32         
 10  MonthYear         11863 non-null  period[M]     
 11  Return            11863 non-null  float64       
 12  Return %          11857 non-null  float64       
 13  Monthly Return    11863 non-null  float64       
 14  Open Value Month  1186

## Covid

In [323]:
file_path = r'C:\Users\michi\OneDrive\MSC DV\Python\Visual Analytics\Data_Covid\owid-covid-data.csv'

df_covid = pd.read_csv(file_path, delimiter=',')
df_covid.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


In [324]:
# nicht benötigte Spalten löschen
columns_to_keep = ['iso_code', 'continent', 'location', 'date', 'new_cases', 'new_deaths', 'new_tests']
df_covid.drop(df_covid.columns.difference(columns_to_keep), axis=1, inplace=True)
df_covid.head()

,iso_code,continent,location,date,new_cases,new_deaths,new_tests
0,AFG,Asia,Afghanistan,2020-01-03,0.0,0.0,NaN
1,AFG,Asia,Afghanistan,2020-01-04,0.0,0.0,NaN
2,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,NaN
3,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,NaN
4,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,NaN


In [325]:
# Spalten umbenennen
df_covid.columns = df_covid.columns.str.replace('iso_code', 'Country short')
df_covid.columns = df_covid.columns.str.replace('continent', 'Continent')
df_covid.columns = df_covid.columns.str.replace('location', 'Country')
df_covid.columns = df_covid.columns.str.replace('date', 'Date')
df_covid.columns = df_covid.columns.str.replace('new_cases', 'Cases')
df_covid.columns = df_covid.columns.str.replace('new_deaths', 'Deaths')
df_covid.columns = df_covid.columns.str.replace('new_tests', 'Tests')

# Datum formatieren
df_covid['Date'] = pd.to_datetime(df_covid['Date'], format='%Y-%m-%d')

df_covid.head()

,Country short,Continent,Country,Date,Cases,Deaths,Tests
0,AFG,Asia,Afghanistan,2020-01-03,0.0,0.0,NaN
1,AFG,Asia,Afghanistan,2020-01-04,0.0,0.0,NaN
2,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,NaN
3,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,NaN
4,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,NaN


In [326]:
df_covid.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344798 entries, 0 to 344797
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Country short  344798 non-null  object        
 1   Continent      328414 non-null  object        
 2   Country        344798 non-null  object        
 3   Date           344798 non-null  datetime64[ns]
 4   Cases          335338 non-null  float64       
 5   Deaths         335378 non-null  float64       
 6   Tests          75403 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 18.4+ MB


# 3. Diagramme

In [327]:
def heatmap():
    # Kennzahlen erstellen
    df_heatmap = df_stocks.pivot(index='Stock/Index', columns='Date', values='Return %')

    # Heatmap erstellen
    fig = px.imshow(df_heatmap,
                    labels=dict(x='Date', y='Stock/Index', color='Return %'),
                    x=df_heatmap.columns,
                    y=df_heatmap.index,
                    color_continuous_scale='RdYlGn',
                    color_continuous_midpoint=0,
                    zmin=-5,
                    zmax=5,
                    )

    # Layout
    fig.update_layout(
        title='Return % Heatmap for Stocks/Indices',
        width=800,
        xaxis_title='Date',
        xaxis=dict(rangeslider_visible=True,showspikes=True, spikethickness=2),
        yaxis_title='Stock/Index',
        coloraxis_colorbar=dict(thicknessmode="pixels", thickness=20, lenmode="pixels", len=250, yanchor="top", y=1, dtick=2),
        autosize=True,
    )

    return fig

heatmap()

In [393]:
def treemap():
    # Kennzahlen erstellen
    df_treemap = df_stocks_reset.groupby('Stock/Index').agg(
        Open=('Open', 'first'),
        Close=('Close', 'last'),
        Volume=('Volume', 'sum')
        ).reset_index()

    df_treemap['Return'] = df_treemap['Close'] - df_treemap['Open']
    df_treemap['Return %'] = ((df_treemap['Return'] / df_treemap['Open']) * 100).round(1)

    # Treemap erstellen
    fig = px.treemap(df_treemap,
                     path=['Stock/Index'],
                     values='Return', # Volumen nehmen, geht aber noch nicht
                     color='Return %',
                     color_continuous_scale='RdYlGn',
                     color_continuous_midpoint=0,
                     title='Treemap: Volume and Return % by Stock/Index')

    # Layout
    fig.update_layout(
        margin = dict(t=50, l=25, r=25, b=25),
        font=dict(size=14, color='black'),
    )

    fig.update_coloraxes(showscale=False) # Farbskala ausblenden

    # Texte
    return_text = ('Return: ' +df_treemap['Return'].round().apply(lambda x: '{:,.0f}'.format(x)).astype(str) + '\n(' + df_treemap['Return %'].astype(str) + '%)') # auf Volumen anpassen
    fig.update_traces(text=return_text, selector=dict(type='treemap'), textposition='middle center', insidetextfont=dict(size=20))

    return fig

treemap()

In [330]:
df_treemap = df_stocks_reset.groupby('Stock/Index').agg(
        Open=('Open', 'first'),
        Close=('Close', 'last'),
        Volume=('Volume', 'sum')
        ).reset_index()

df_treemap['Return'] = df_treemap['Close'] - df_treemap['Open']
df_treemap['Return %'] = (df_treemap['Return'] / df_treemap['Open']) * 100

df_treemap

,Stock/Index,Open,Close,Volume,Return,Return %
0,BTC-USD,6603.640137,26342.904297,-3923452624896,19739.264160,298.914898
1,CAC 40 (^FCHI),5468.910156,7184.819824,101741102500,1715.909668,31.375715
2,CMC Crypto 200 Index by Solactive (^CMC200),100.000000,566.789001,0,466.789001,466.789001
3,DAX PERFORMANCE-INDEX (^GDAXI),12341.849609,15557.290039,102949602400,3215.440430,26.053149
4,EURUSD=X,1.175323,1.059547,0,-0.115776,-9.850569
5,GBPUSD=X,1.311716,1.221195,0,-0.090521,-6.900960
6,HANG SENG INDEX (^HSI),27606.810547,17729.289063,164444735140,-9877.521484,-35.779292
7,NASDAQ Composite (^IXIC),8001.589844,13211.809570,-2104144832928,5210.219726,65.114806
8,Nikkei 225 (^N225),23881.849609,32678.619141,89651700000,8796.769532,36.834540
9,Russell 2000 (^RUT),1705.489990,1776.500000,-2669927356352,71.010010,4.163613


In [385]:
def liste_stocks():





liste_stocks()

0        6603.640137
1        6452.790039
2        6495.290039
3        6678.750000
4        6643.100098
            ...     
11858     147.604996
11859     147.742004
11860     148.320999
11861     147.643997
11862     148.360001
Name: Open, Length: 11863, dtype: float64

In [ ]:
def liste_covid():





liste_covid()